---
layout: post
title: "Escape Room 3.15 — Level 3 Simulation Builder"
description: "Final lock - assemble the RANDOM + IF/ELSE blocks to simulate a weighted spinner and verify the results by running trials"
type: lesson
comments: false
permalink: /3.15/03-random-simulator
---


# 🗝️ Level 3 — Simulation Builder

The last lock only opens if you **build** the logic yourself.  
Drag the blocks to assemble the algorithm that simulates the spinner below:

- The spinner has **8 equal slices**  
  - **3** green  
  - **2** blue  
  - **1** each: red, orange, purple

When your blocks are in the right order, run the simulation to see the empirical percentages.


In [5]:
%%html 

<div id="l3-sim" style="margin:18px 0; padding:16px; border:1px solid rgba(255,255,255,.12); border-radius:14px; background:linear-gradient(180deg,#101635,#0b1026);">
  <h3 style="margin:0 0 12px;">🔒 Final Lock — Build the Spinner Algorithm</h3>
  <p style="margin:6px 0 12px; color:#cfe7ff">
    Drag the blocks into the **Assembly Area** (top → bottom). Then press <em>Check Structure</em>. If correct, you can run the simulation.
  </p>

  <div style="display:grid; gap:12px; grid-template-columns:1fr 1fr;">
    <!-- Palette -->
    <div>
      <h4 style="margin:0 0 8px;">🧰 Block Palette</h4>
      <div id="palette" style="min-height:240px; padding:10px; border-radius:10px; background:rgba(255,255,255,.04); display:grid; gap:8px;">
        <!-- Blocks injected by JS -->
      </div>
      <button id="reset" style="margin-top:10px;">↺ Reset Blocks</button>
    </div>

    <!-- Assembly -->
    <div>
      <h4 style="margin:0 0 8px;">🧱 Assembly Area (top executes first)</h4>
      <div id="assembly" style="min-height:240px; padding:10px; border-radius:10px; background:rgba(255,255,255,.06); outline:2px dashed rgba(255,255,255,.18); display:grid; gap:8px;"></div>

      <div style="display:flex; gap:8px; margin-top:10px;">
        <button id="check">✅ Check Structure</button>
        <button id="run" disabled>▶ Run 10,000 Spins</button>
      </div>

      <div id="feedback" style="margin-top:8px; font-weight:600;"></div>
      <div id="results" style="margin-top:10px;"></div>
    </div>
  </div>

  <style>
  /* General text inside the activity */
  #l3-sim, 
  #l3-sim h3, 
  #l3-sim h4, 
  #l3-sim p, 
  #l3-sim code, 
  #l3-sim div, 
  #l3-sim span {
    color: #e9eefc; /* bright off-white */
  }

  /* Buttons */
  #l3-sim button {
    padding:6px 10px;
    border-radius:6px;
    border:1px solid rgba(255,255,255,.25);
    background:rgba(255,255,255,.06);
    color:#e9eefc;
    cursor:pointer;
  }
  #l3-sim button:hover {
    background: rgba(255,255,255,.12);
  }
  #l3-sim button[disabled] {
    opacity:.5; 
    cursor:not-allowed;
  }

  /* Blocks */
  #l3-sim .block {
    user-select:none; cursor:grab; padding:8px 10px; border-radius:8px;
    border:1px solid rgba(255,255,255,.18); 
    background:rgba(255,255,255,.08);
    color:#e9eefc;
    font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", monospace;
    display:flex; align-items:center; gap:8px;
  }
  #l3-sim .block:active { cursor:grabbing; }

  /* Block tag badges */
  #l3-sim .tag {
    font-size:.75rem; 
    padding:2px 6px; 
    border-radius:999px; 
    background:rgba(255,255,255,.15);
    color:#cfe7ff;
  }

  /* Feedback */
  #l3-sim .ok { color:#7CFC00; }
  #l3-sim .bad { color:#FF6B6B; }

  /* Results pills */
  #l3-sim .pill { 
    display:inline-block; 
    padding:2px 8px; 
    border-radius:999px; 
    background:rgba(255,255,255,.12); 
    color:#e9eefc;
    margin-right:6px; 
  }

  /* Indentation for nested-style blocks */
  #l3-sim .indent-1 { margin-left:16px; }
  #l3-sim .indent-2 { margin-left:32px; }
</style>


  <script>
  (function(){
    // ---- Target algorithm order (IDs must match):
    // 1) assign, 2) ifG, 3) elifB, 4) elifR, 5) elifO, 6) elseP
    const EXPECT = ["assign","ifG","elifB","elifR","elifO","elseP"];

    const BLOCKS = [
      { id:"assign", text:"spin ← RANDOM(1,8)", tag:"SET", indent:0 },
      { id:"ifG",   text:"IF spin ≤ 3 ➜ DISPLAY(\"Green\")", tag:"IF", indent:0 },
      { id:"elifB", text:"ELSE IF (spin = 4 OR spin = 5) ➜ DISPLAY(\"Blue\")", tag:"ELSE IF", indent:0 },
      { id:"elifR", text:"ELSE IF spin = 6 ➜ DISPLAY(\"Red\")", tag:"ELSE IF", indent:0 },
      { id:"elifO", text:"ELSE IF spin = 7 ➜ DISPLAY(\"Orange\")", tag:"ELSE IF", indent:0 },
      { id:"elseP", text:"ELSE ➜ DISPLAY(\"Purple\")", tag:"ELSE", indent:0 },
      // Decoys to encourage thinking
      { id:"decoy1", text:"IF spin < 3 ➜ DISPLAY(\"Green\")", tag:"IF", indent:0 },
      { id:"decoy2", text:"ELSE IF spin ≥ 6 ➜ DISPLAY(\"Purple\")", tag:"ELSE IF", indent:0 }
    ];

    const palette = document.getElementById("palette");
    const assembly = document.getElementById("assembly");
    const feedback = document.getElementById("feedback");
    const results = document.getElementById("results");
    const checkBtn = document.getElementById("check");
    const runBtn = document.getElementById("run");
    const resetBtn = document.getElementById("reset");

    // Create a draggable block element
    function makeBlock(b){
      const el = document.createElement("div");
      el.className = "block " + (b.indent?("indent-"+b.indent):"");
      el.draggable = true;
      el.dataset.blockId = b.id;
      el.innerHTML = `<span class="tag">${b.tag}</span><span>${b.text}</span>`;
      el.addEventListener("dragstart", ev=>{
        ev.dataTransfer.setData("text/plain", b.id);
        // allow move within assembly: mark source
        ev.dataTransfer.setData("from", el.parentElement.id);
      });
      return el;
    }

    function renderPalette(){
      palette.innerHTML = "";
      BLOCKS.forEach(b => palette.appendChild(makeBlock(b)));
    }
    function renderAssemblyPlaceholders(){
      assembly.innerHTML = "";
      // Allow any number of blocks; order matters
      assembly.addEventListener("dragover", ev=>ev.preventDefault());
      assembly.addEventListener("drop", onDropAssembly);
    }

    function onDropAssembly(ev){
      ev.preventDefault();
      const id = ev.dataTransfer.getData("text/plain");
      const from = ev.dataTransfer.getData("from");
      // If dragged from palette, clone; if from assembly, move
      if(!id) return;
      // If block is already present and came from palette, allow duplicates? -> No.
      if(from==="palette" && [...assembly.querySelectorAll(".block")].some(el=>el.dataset.blockId===id)){
        // prevent duplicates for core blocks
        return;
      }
      const blockDef = BLOCKS.find(b=>b.id===id);
      if(!blockDef) return;
      if(from==="assembly"){
        // Move existing node to the end (simple behavior)
        const node = [...assembly.children].find(el=>el.dataset.blockId===id);
        if(node){ assembly.removeChild(node); assembly.appendChild(node); }
      } else {
        assembly.appendChild(makeBlock(blockDef));
      }
      feedback.textContent = "";
      results.innerHTML = "";
      runBtn.disabled = true;
    }

    // Allow dropping back into palette to remove from assembly
    palette.addEventListener("dragover", ev=>ev.preventDefault());
    palette.addEventListener("drop", ev=>{
      ev.preventDefault();
      const id = ev.dataTransfer.getData("text/plain");
      const from = ev.dataTransfer.getData("from");
      if(from==="assembly"){
        const node = [...assembly.children].find(el=>el.dataset.blockId===id);
        if(node) assembly.removeChild(node);
      }
      feedback.textContent = "";
      results.innerHTML = "";
      runBtn.disabled = true;
    });

    // Structure checker
    checkBtn.onclick = ()=>{
      const ids = [...assembly.querySelectorAll(".block")].map(el=>el.dataset.blockId);
      const exact = JSON.stringify(ids) === JSON.stringify(EXPECT);
      if(ids.length === 0){
        feedback.innerHTML = `<span class="bad">Add blocks to the Assembly Area first.</span>`;
        runBtn.disabled = true;
        return;
      }
      if(exact){
        feedback.innerHTML = `<span class="ok">Perfect structure! You can now run the simulation.</span>`;
        runBtn.disabled = false;
      } else {
        // Give a targeted hint
        let hint = "";
        if(!ids.includes("assign")) hint = "Start with <code>spin ← RANDOM(1,8)</code>.";
        else if(ids[0] !== "assign") hint = "The assignment block must be first.";
        else if(!ids.includes("ifG")) hint = "You need the IF for Green (≤ 3).";
        else if(ids.indexOf("elifB") < ids.indexOf("ifG")) hint = "Blue check must come after Green.";
        else hint = "Order should be: assign → IF Green → ELSE IF Blue → ELSE IF Red → ELSE IF Orange → ELSE Purple.";
        feedback.innerHTML = `<span class="bad">Not quite.</span> ${hint}`;
        runBtn.disabled = true;
      }
      results.innerHTML = "";
    };

    // Simulation: 10k spins, show empirical percentages
    runBtn.onclick = ()=>{
      const trials = 10000;
      const counts = {Green:0, Blue:0, Red:0, Orange:0, Purple:0};
      for(let i=0;i<trials;i++){
        const spin = Math.floor(Math.random()*8)+1; // 1..8
        let color = "";
        if(spin <= 3) color = "Green";
        else if (spin===4 || spin===5) color = "Blue";
        else if (spin===6) color = "Red";
        else if (spin===7) color = "Orange";
        else color = "Purple";
        counts[color]++;
      }
      const pct = c => (c/trials*100).toFixed(1)+"%";
      results.innerHTML = `
        <div><span class="pill">Green</span> ${pct(counts.Green)} (target 37.5%)</div>
        <div><span class="pill">Blue</span> ${pct(counts.Blue)} (target 25.0%)</div>
        <div><span class="pill">Red</span> ${pct(counts.Red)} (target 12.5%)</div>
        <div><span class="pill">Orange</span> ${pct(counts.Orange)} (target 12.5%)</div>
        <div><span class="pill">Purple</span> ${pct(counts.Purple)} (target 12.5%)</div>
        <div style="margin-top:8px; opacity:.8;">(Targets are slices/8: 3/8, 2/8, 1/8, 1/8, 1/8)</div>
      `;
    };

    resetBtn.onclick = ()=>{
      renderPalette();
      renderAssemblyPlaceholders();
      feedback.textContent = "";
      results.innerHTML = "";
      runBtn.disabled = true;
    };

    // Initial render
    renderPalette();
    renderAssemblyPlaceholders();
  })();
  </script>
</div>



---

<div style="margin-top:2rem; text-align:center;">
  <a href="/3.15/homepage" style="
    display:inline-block;
    padding:10px 18px;
    background:#1a2148;
    color:#e9eefc;
    border-radius:12px;
    text-decoration:none;
    box-shadow:0 4px 10px rgba(0,0,0,.3);
    transition:all .2s ease;">
    ⬅️ Back to Lobby
  </a>
</div>
